## Train the model based Using YOLOv8:

In [2]:
##first let's import our library 
import ultralytics
from ultralytics import YOLO

model = YOLO('yolov8.pt')  # load the pretraind model or we can simply use this command for training from scratch 'model = YOLO('yolov8.yaml')
model.train(data="path/to/yaml",val=False,epochs=300) # start training ! specify the parameters as you want epochs , batch seze,...

## validate your Model

In [ ]:
results = model.val(data='path/to/yaml') ##Validate your model 

## Feature extraction :

In [ ]:
import torch
from torchvision import transforms
from PIL import Image, ImageDraw
from pathlib import Path
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import cv2
import io



In [4]:
def hook_fn(module, input, output):
    intermediate_features.append(output)

# Define feature extraction function
def extract_features(model, img, layer_index=22):
    global intermediate_features
    intermediate_features = []
    hook = model.model.model[layer_index].cv3[1][2].register_forward_hook(hook_fn) ## Here you can modify ti choose which layer you want to extract ie hook = model.model.model[layer_index].register_forward_hook(hook_fn)
    with torch.no_grad():
        model(img)
    hook.remove()
    return intermediate_features[0]  # Access the first element of the list

def preprocess_image(img_path):
    transform = transforms.Compose([
        transforms.Resize((640, 640)),
        transforms.Grayscale(num_output_channels=3),  # Convert to RGB
        transforms.ToTensor(),
        transforms.Normalize(mean=0., std=1.)
    ])
    img = Image.open(img_path)
    img = transform(img)
    img = img.unsqueeze(0)
    return img



img = 'path/to/image'
model = YOLO('path/to/best/weights/on/folder/best.pt')
img = preprocess_image(img)
feature_extracted = extract_features(img,model,layer_index= 12) #set on perfer feature


## Plot the feature extracted:

In [ ]:
def extract_and_plot_features(image_dir, layer_index=21, channel_index=5):
    for img_path in Path(image_dir).glob('*.jpg'):
        img = preprocess_image(img_path)
        features = extract_features(model, img, layer_index)

        # Print the shape of the features
        print(f"Features shape for {img_path.name}: {features.shape}")

        # Plot the features as a heatmap for a specific channel
        plt.figure(figsize=(10, 5))
        sns.heatmap(features[0][channel_index].cpu().numpy(), cmap='viridis', annot=False)
        plt.title(f'Features for {img_path.name} - Layer {layer_index} - Channel {channel_index}')
        plt.show()